### Imports

In [4]:
from chelombus import DataStreamer
from chelombus import FingerprintCalculator


### FINGERPRINT MODULE

Simple example on how the API works

In [5]:
# Define a list of SMILES strings
smiles_list = ["CCO", "C1CCCCC1", "O=C=O", "O=C=O"]

# Define fingerprint parameters
params = {'fpSize': 1024, 'radius': 2}

# Create an instance of FingerprintCalculator
calculator = FingerprintCalculator()

# Compute fingerprints for the list of SMILES strings
fingerprints = calculator.FingerprintFromSmiles(smiles_list, 'morgan', **params)

# Display the shape of the output fingerprint array
print(f"Fingerprints shape: {fingerprints.shape}")
print("Fingerprint 1", fingerprints[1])


Fingerprints shape: (4, 1024)
Fingerprint 1 [0 0 1 ... 0 0 0]


How to use the API for loading fingerprints in chunks

In [6]:
# Import iterator method
ds = DataStreamer()

chunksize = 10_000
smiles= ds.parse_input(input_path="smiles_examples.smi", chunksize=chunksize, smiles_col=1)
print(type(smiles)) # This is only the generator, in order to get each chunk of data we need to iterate

count = 0
for smiles_chunk in smiles:
     count += len(smiles_chunk)
     calculator.FingerprintFromSmiles(smiles_chunk, 'morgan', **params)
     print(f"\r Fingerprints calculated: {count:,}", end='', flush=True)


<class 'generator'>
 Fingerprints calculated: 100,000

If we want to save each chunk as a separate file -ideal for large chunks that we could use later- then `save_chunk` from the `helper_functions.py`is provided

In [7]:
from chelombus.utils.helper_functions import save_chunk

smiles= ds.parse_input(input_path="smiles_examples.smi", chunksize=chunksize, smiles_col=1)

count = 0
for idx, smiles_chunk in enumerate(smiles):
    count += len(smiles_chunk)
    fp_chunk = calculator.FingerprintFromSmiles(smiles_chunk, 'morgan', **params)
    save_chunk(fp_chunk, output_dir="data", chunk_index=idx, file_format='npy')
    print(f"\r Fingerprints calculated: {count:,}", end='', flush=True)


 Fingerprints calculated: 100,000

### Product Quantization

In [8]:
# At this step we could load Fingerprints computed in the previous step -likely what you'd do for large datasets
# or in case it fits in memory just pass the fingerprint matrix
import pandas as pd

df = pd.read_csv("smiles_examples.smi", sep="\t", header=None, names=["id", "smiles"])
smiles = df["smiles"].tolist()
fingerprints = calculator.FingerprintFromSmiles(smiles, 'morgan', fpSize=1024, radius=3)

print(fingerprints.shape, f"{fingerprints.nbytes / 1e6}  MB")


(100000, 1024) 102.4  MB


Then we fit the PQ encoder using our training data. 
`K`refers to the number of centroids to be used when running KMeans on each subvector. 
`m`is the number of subvectors (splits) from our input data. 
`iterations`is the maximum number of iterations each KMeans is going to do. 
With higher `K`and `iterations`, higher training times. 

In [9]:
from chelombus.encoder.encoder import PQEncoder

pq_encoder = PQEncoder(k=256, m=4, iterations=10)
pq_encoder.fit(fingerprints)


Training PQ-encoder: 100%|██████████| 4/4 [00:14<00:00,  3.67s/it]


We can check some atributes: 

`.codewords` are the centroids coordinates gathered from each KMeans run on every subvector. Since we have 4 splits, 256 centroids and the subvectors are of size 1024/4 = 256, then the codebook is shape (4, 256, 256)

After the `pq_encoder` is fitted, the encoder has an attribute to account for the training process. If we try to use transform without fitting we would get an Error. So know, we check that the ecoder was in fact trained. 

If we want to access all the `KMeans`attributes that one would normally get from sklearn, we can do so using the list of KMeans in `pq_trained` and use any attribute you would normally use. Like `.labels_` to check the index of the centroids for each training sample. 

In [10]:
print("The shape of the codebook is: ", pq_encoder.codewords.shape)
print("Is the encoder trained? ", pq_encoder.encoder_is_trained)
print(f"The lables: {pq_encoder.pq_trained[0]} have length: {len(pq_encoder.pq_trained[0].labels_)}")


The shape of the codebook is:  (4, 256, 256)
Is the encoder trained?  True
The lables: KMeans(max_iter=10, n_clusters=256) have length: 100000


After the training process we can create our PQ codes.
The PQCodes are going to be of shape `(Number of samples, m)`. 

In [11]:
import numpy as np 
import time

# First we load and calculate our fingerprints 
X_test = np.load('data/fingerprints_chunk_00000.npy')
print(f"{X_test.shape[0]:,} fingerprints of {X_test.shape[1]} dimensions to be transformed into PQ-codes")

s = time.time()
X_pq_code = pq_encoder.transform(X_test)
e = time.time()
print(f"Transforming {X_test.shape[0]:,} fingeprints took {(e-s):.2f} seconds")


10,000 fingerprints of 1024 dimensions to be transformed into PQ-codes


Generating PQ-codes: 100%|██████████| 4/4 [00:00<00:00, 88.94it/s]

Transforming 10,000 fingeprints took 0.05 seconds


Since we have a PQ-code of 4 digits and each digit can take value {0,255} then the number of theoretical unique PQ-codes we can get is therefore $256^4 = 4,294,967,296$. However we can test that this is much less in reality.  

In [12]:
X_pq_code


array([[ 33,  91,  53,  83],
       [ 33, 110, 134,  83],
       [ 91,  91,  90, 119],
       ...,
       [ 70,  47, 199,  96],
       [ 70,  47, 156,  18],
       [228,  31, 156, 204]], shape=(10000, 4), dtype=uint8)

In [13]:
import numpy as np
# Count unique rows
unique_rows = np.unique(X_pq_code, axis=0)
num_unique_vectors = unique_rows.shape[0]
print("Number of unique 4-dim vectors:", num_unique_vectors)


Number of unique 4-dim vectors: 6273


The main advantage of transforming the binary fingerprints into PQ-codes is that we are storing (almost) the same information in a much more efficient way. We can check that the amount of memory required to store the same data is 256x times less

In [14]:
print(f"Original input of shape: {X_test.shape} and size of {X_test.nbytes:,} bytes is now transformed into shape {X_pq_code.shape} and size of {X_pq_code.nbytes:,} bytes")
print(f"This is {int(X_test.nbytes / X_pq_code.nbytes)} times more memory efficient")


Original input of shape: (10000, 1024) and size of 10,240,000 bytes is now transformed into shape (10000, 4) and size of 40,000 bytes
This is 256 times more memory efficient


### PQKMeans Clustering

After encoding data into PQ-codes, we can perform efficient clustering directly on the compressed representation using PQKMeans.

In [15]:
from chelombus import PQKMeans

# Create a PQKMeans clusterer
# k: number of clusters
# iteration: number of k-means iterations
n_clusters = 100
clusterer = PQKMeans(encoder=pq_encoder, k=n_clusters, iteration=20, verbose=False)

# Fit and predict in one step
labels = clusterer.fit_predict(X_pq_code)
print(f"Assigned {len(labels)} molecules to {n_clusters} clusters")
print(f"Labels shape: {labels.shape}")
print(f"Sample labels: {labels[:10]}")


Assigned 10000 molecules to 100 clusters
Labels shape: (10000,)
Sample labels: [47 76 36 36 36 94 94 94 57 94]


We can also access cluster centers (as PQ codes) and check cluster distribution.

In [16]:
# Cluster centers are stored as PQ codes
print(f"Cluster centers shape: {clusterer.cluster_centers_.shape}")

# Check cluster distribution
unique, counts = np.unique(labels, return_counts=True)
print(f"\nCluster size statistics:")
print(f"  Min cluster size: {counts.min()}")
print(f"  Max cluster size: {counts.max()}")
print(f"  Mean cluster size: {counts.mean():.1f}")


Cluster centers shape: (100, 4)

Cluster size statistics:
  Min cluster size: 1
  Max cluster size: 304
  Mean cluster size: 102.0


You can also use `fit()` and `predict()` separately, useful when assigning new data to existing clusters.

In [17]:
# Fit on training data
clusterer2 = PQKMeans(encoder=pq_encoder, k=50, iteration=15)
clusterer2.fit(X_pq_code)

# Predict on new data (using same data for demo)
new_labels = clusterer2.predict(X_pq_code)
print(f"Predicted labels: {new_labels[:10]}")


Predicted labels: [16 16 36 36 36 10 10 10 18 33]


### Saving and Loading Models

Both the PQEncoder and PQKMeans can be saved and loaded for later use.

In [18]:
# Save the trained clusterer
clusterer2.save("data/pqkmeans_model.joblib")

# Load the model later
loaded_clusterer = PQKMeans.load("data/pqkmeans_model.joblib")

# Use the loaded model to predict
loaded_labels = loaded_clusterer.predict(X_pq_code)
print(f"Loaded model predictions match: {np.array_equal(new_labels, loaded_labels)}")


Loaded model predictions match: True


### Cluster I/O

The `cluster_io` module provides utilities to query and export clustered data using DuckDB. This is useful for large-scale datasets stored as chunked parquet files.

In [19]:
# First, let's create sample parquet files for demonstration
import os

os.makedirs("data/results", exist_ok=True)

# Create a dataframe with smiles and cluster_id
cluster_df = pd.DataFrame({
    "smiles": smiles[:len(labels)],
    "cluster_id": labels
})

# Save as chunked parquet (simulating pipeline output)
chunk_size = 5000
for i, start in enumerate(range(0, len(cluster_df), chunk_size)):
    chunk = cluster_df.iloc[start:start + chunk_size]
    chunk.to_parquet(f"data/results/chunk_{i:05d}.parquet", index=False)

print(f"Created {len(os.listdir('data/results'))} parquet chunks")


Created 2 parquet chunks


In [20]:
from chelombus import get_cluster_stats, query_cluster, sample_from_cluster

# Get statistics for all clusters
stats = get_cluster_stats("data/results/")
print(f"Number of clusters: {len(stats)}")
print(f"Total molecules: {stats['molecule_count'].sum():,}")
print(f"\nTop 5 largest clusters:")
print(stats.nlargest(5, 'molecule_count'))


Number of clusters: 98
Total molecules: 10,000

Top 5 largest clusters:
    cluster_id  molecule_count
11          11             304
36          36             281
78          79             276
90          91             251
54          54             250


In [23]:
# Query all molecules from a specific cluster
cluster_id = stats.iloc[0]['cluster_id']  # Get largest cluster
cluster_data = query_cluster("data/results/", cluster_id=int(cluster_id))
print(f"Cluster {cluster_id} contains {len(cluster_data)} molecules")
print(cluster_data.head())


Cluster 0 contains 74 molecules
                                              smiles  cluster_id
0  CC(C1=NN=C(SCCC2=CC=NC=C2)N1C1=CC=C(F)C=C1)N1C...           0
1  CC(C1=NN=C(SCC2CC(=O)N(C)C2)N1C1=CC=C(F)C=C1)N...           0
2  CC(C1=NN=C(SCC2=C(Br)N(C)N=C2)N1C1=CC=C(F)C=C1...           0
3  CC(C1=NN=C(SC[C@H]2C[C@H]3C=C[C@@H]2O3)N1C1=CC...           0
4  CC(C1=NN=C(SCC(=O)C2=C(F)C=CC=C2F)N1C1=CC=C(F)...           0


In [24]:
# Get a random sample from a cluster (useful for previewing)
sample = sample_from_cluster("data/results/", cluster_id=int(cluster_id), n=5, random_state=42)
print(f"Random sample of 5 molecules from cluster {cluster_id}:")
print(sample['smiles'].tolist())


Random sample of 5 molecules from cluster 0:
['FC1=CC=C(N2C(CN3CCCCC3)=NN=C2SCC2=CC=NC(Cl)=C2F)C=C1', 'O=C(CSC1=NN=C(CN2CCCCC2)N1C1=CC=CC=C1)C1=CC=CNC1=O', 'CC(C1=NN=C(SCC2=CC=C(C(=O)O)C(F)=C2)N1C1=CC=C(Cl)C=C1)N(C)C', 'COC1=CC(CSC2=NN=C(CN3CCCCC3)N2C2=CC=C(F)C=C2)=NC=N1', 'CC(C)N(C)C[C@H](O)CSC1=NN=C(CN2CCCCC2)N1C1=CC=C(F)C=C1']


In [25]:
from chelombus import export_cluster

# Export a single cluster to a file
count = export_cluster(
    "data/results/", 
    cluster_id=int(cluster_id), 
    output_path="data/cluster_export.parquet"
)
print(f"Exported {count} molecules to data/cluster_export.parquet")


Exported 74 molecules to data/cluster_export.parquet


### Visualization with TMAP

Create interactive tree maps (TMAPs) to visualize molecular clusters. TMAPs use LSH Forest to find nearest neighbors and create a tree-like layout.

Note: Because TMAP is outdated it is not compatible with Python versions above 3.8. I am working on creating a new version of TMAP so in the meantime if you wish to create the TMAPs you could 

- a downgrade the version to 3.8

- b install tmap in a separate environment and use that one when generating the maps. 

Sorry for the inconvenience!

In [26]:
from chelombus.utils.visualization import create_tmap
# Create a TMAP for a subset of molecules (for demonstration)
# Note: For large datasets, create TMAPs per cluster
sample_smiles = smiles[:1000]

# This will generate an interactive HTML file
create_tmap(
    smiles=sample_smiles,
    fingerprint='morgan',
    tmap_name='sample_tmap'
)
print("TMAP generated: sample_tmap_TMAP.html")


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/afloresep/work/Chelombus/venv/lib/python3.12/site-packages/faerun/faerun.py:409: RuntimeWarning: invalid value encountered in divide
  data_c[s] = (data_c[s] - min_c[s]) / (max_c[s] - min_c[s])


/home/afloresep/work/Chelombus/examples/sample_tmap_TMAP.html

TMAP generated: sample_tmap_TMAP.html


### Summary

This tutorial covered the main Chelombus API:

1. **Fingerprint Calculation**: Use `FingerprintCalculator` to compute molecular fingerprints
2. **Data Streaming**: Use `DataStreamer` to process large files in chunks
3. **Product Quantization**: Use `PQEncoder` to compress fingerprints into compact PQ-codes
4. **Clustering**: Use `PQKMeans` to cluster PQ-codes efficiently
5. **Cluster I/O**: Use `query_cluster`, `export_cluster`, `get_cluster_stats` etc. for querying clustered data
6. **Visualization**: Use `create_tmap` to generate interactive visualizations

For billion-scale datasets, combine these components in a streaming pipeline:
- Stream SMILES in chunks
- Compute fingerprints per chunk
- Transform to PQ-codes
- Cluster with PQKMeans
- Export results as parquet
- Visualize per-cluster TMAPs